# Contextual Embeddings with ELMo
ELMo learns contextualized word vectors by running the text through a deep recurrent network.  
ELMo is actually an algorithm for unsupervised learning and does not make any use of the labels we have for our text classification task. The authors do show that contextualized word vectors obtained using ELMo increase text classification performance in a large array of tasks. Let's see if we see a significant gain in our case!  

There are good examples of using ELMo in both [the AllenNLP github repo](https://github.com/allenai/allennlp/blob/master/tutorials/how_to/elmo.md) and [this AnalyticsVidhya post](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/?utm_source=blog&utm_medium=top-pretrained-models-nlp-article). In this guide we'll use the python package [Flair](https://github.com/zalandoresearch/flair) to get ELMo embeddings.

## Contextual word embeddings with ELMo in Flair
In Flair, you init a `Sentence` object given the tokens seperated by spaces.  
Sentence has a few useful attributes and methods

In [ ]:
!pip install allennlp
!pip install flair

In [1]:
from flair.embeddings import Sentence
sentence = Sentence('The grass is green .')

We also init a class of the desired embedding method. The `embed` method of this class gets a Sentence and adds to its tokens the relevant embedding.

In [2]:
from flair.embeddings import ELMoEmbeddings

# init embedding
elmo_embedding = ELMoEmbeddings()

elmo_embedding.embed(sentence)
for token in sentence:
    print(token)
    print(token.embedding.shape)
    print(token.embedding)

Token: 1 The
torch.Size([3072])
tensor([-0.3288,  0.2022, -0.5940,  ..., -1.2773,  0.3049,  0.2150])
Token: 2 grass
torch.Size([3072])
tensor([ 0.2539, -0.2363,  0.5263,  ..., -0.7001,  0.8798,  1.4191])
Token: 3 is
torch.Size([3072])
tensor([ 0.1915,  0.2300, -0.2894,  ..., -0.3626,  1.9066,  1.4520])
Token: 4 green
torch.Size([3072])
tensor([ 0.1779,  0.1309, -0.1041,  ..., -0.1006,  1.6152,  0.3299])
Token: 5 .
torch.Size([3072])
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.0106, -0.0833,  0.0669])


**Try it yourself:** Now, compare the embeddings obtained using ELMo for the same word in different contexts. Are they equal or different?

### Word sense disambiguation using ELMo
**Try it yourself:** Let's also try to see how ELMo handles word sense disambiguation. Below are 6 sentences with 2 different meanings of the word `bank`. Try to see if ELMo vectors indeed separate the two meanings.

In [3]:
sentences = [
    "I was walking along the river bank",
    "I saw a toad near the east bank of the river",
    "We had a nice picnic by the bank",
    "I need to deposit money from the bank",
    "The bank branch is closed",
    "He started working at the bank"
]

## Sentence Embedding and Text Classification

### Classify documents using the average of contextual word vectors
**Try it yourself:** *Optional:* In previous sections we've built a classifier using the average of non-contextual word vectors. Now, try to use contextual word embeddings on our dataset. Use the average of these vectors and apply a classifier on it to obtain the predictions. Is the performance better than for non-contextual word vectors?

### Sentence embedding in Flair

#### By Pooling Word Vectors

We've used Flair to get embeddings for each word in the sentence. However, for text classification of the entire document, we need a way to integrate all these vectors into a single document embedding. There are several methods for that, and those interested would find this article useful - https://towardsdatascience.com/document-embedding-techniques-fed3e7a6a25d

The most basic element is averaging the word embedding into a single document embedding. In FLAIR, we do this using a DocumentPooling.

In [4]:
from flair.embeddings import DocumentPoolEmbeddings
document_embeddings = DocumentPoolEmbeddings([elmo_embedding], pooling='mean')

document_embeddings.embed(sentence)
document_embeddings

DocumentPoolEmbeddings(
  fine_tune_mode=linear, pooling=mean
  (embeddings): StackedEmbeddings(
    (list_embedding_0): ELMoEmbeddings(model=elmo-original)
  )
  (embedding_flex): Linear(in_features=3072, out_features=3072, bias=False)
)

In [5]:
# now check out the embedded sentence.
print(sentence.get_embedding().shape)
print(sentence.get_embedding())

torch.Size([3072])
tensor([-0.1185,  0.0253, -0.3043,  ..., -0.4902,  0.9246,  0.6966],
       grad_fn=<CatBackward>)


#### By using an RNN
Alternatively, we can use an RNN that runs over the word embeddings. We will use the last hidden state as the document embedding. In this case it is very helpful to train the model using the true labels of our task, so that the RNN is optimized for our own data and task:

In [6]:
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings
glove_embedding = WordEmbeddings('glove')
document_embeddings = DocumentRNNEmbeddings([glove_embedding], rnn_type='LSTM')

In [7]:
# create an example sentence
sentence = Sentence('The grass is green . And the sky is blue .')

# embed the sentence with our document embedding
document_embeddings.embed(sentence)

# now check out the embedded sentence.
print(sentence.get_embedding())

tensor([ 0.1272, -0.2822, -0.0956,  0.1956,  0.0457,  0.1642, -0.0536,  0.1531,
         0.1508,  0.0272, -0.0358, -0.0921, -0.0925, -0.0604, -0.0436,  0.0267,
        -0.1363,  0.0222,  0.2765, -0.1669,  0.3440, -0.1686,  0.0097,  0.0951,
        -0.1302,  0.0935,  0.0545, -0.1169, -0.0216,  0.1105,  0.0826, -0.0503,
         0.1294, -0.0573,  0.0587, -0.1517,  0.1281, -0.1687, -0.1632, -0.0288,
        -0.1537,  0.1496,  0.0245, -0.0163,  0.0061, -0.0661, -0.1161, -0.1996,
        -0.0632,  0.0372,  0.0977, -0.0737,  0.1587,  0.1859,  0.0448, -0.0353,
        -0.1446, -0.2075, -0.1587, -0.1796,  0.1638,  0.0912,  0.1589,  0.0454,
        -0.1820, -0.0242,  0.2046, -0.1038, -0.0801,  0.0520,  0.0825,  0.0676,
         0.0445, -0.2015,  0.2772,  0.0453, -0.1087,  0.0425,  0.0335, -0.0324,
         0.0267,  0.0946,  0.0690,  0.1318,  0.0240,  0.1387,  0.0422,  0.0966,
         0.1493, -0.0437,  0.1057, -0.0929,  0.1106,  0.0138, -0.0067,  0.1094,
        -0.0695,  0.1147,  0.1213,  0.04

Note that while `DocumentPoolEmbeddings` are immediately meaningful, `DocumentRNNEmbeddings` need to be tuned on the downstream task. This happens automatically in Flair if you train a new model with these embeddings. You can find an example of training a text classification model [here](/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md#training-a-text-classification-model). Once the model is trained, you can access the tuned DocumentRNNEmbeddings object directly from the classifier object and use it to embed sentences.

`DocumentRNNEmbeddings` have a number of hyper-parameters that can be tuned to improve learning:

```
:param hidden_size: the number of hidden states in the rnn.
:param rnn_layers: the number of layers for the rnn.
:param reproject_words: boolean value, indicating whether to reproject the token embeddings in a separate linear
layer before putting them into the rnn or not.
:param reproject_words_dimension: output dimension of reprojecting token embeddings. If None the same output
dimension as before will be taken.
:param bidirectional: boolean value, indicating whether to use a bidirectional rnn or not.
:param dropout: the dropout value to be used.
:param word_dropout: the word dropout value to be used, if 0.0 word dropout is not used.
:param locked_dropout: the locked dropout value to be used, if 0.0 locked dropout is not used.
:param rnn_type: one of 'RNN' or 'LSTM'
```

#### Training RNN on our own dataset - Loading dataset
The simplest way to load our data in Flair is using a CSV file. You can learn about other method in [the documentation](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md).

To create a `Corpus` for a text classification task, you need to have three files (train, dev, and test) in the 
above format located in one folder. This data folder structure could, for example, look like this for the IMDB task:
```text
/data/train.csv
/data/val.csv
/data/test.txt
```
Now create a `CSVClassificationCorpus` by pointing to this folder (`/data`). 
Thereby, each line in a file is converted to a `Sentence` object annotated with the labels.

Attention: A text in a line can in fact have multiple sentences. Thus, a `Sentence` object is actually a `Document` and can actually consist of multiple sentences.

In [11]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = 'data/'

# column format indicating which columns hold the text and label(s). This is 1-based and not 0-based
column_name_map = {3: "text", 2: "label"}

# load corpus containing training, test and dev data
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                      test_file='test.csv',
                                      dev_file='val.csv',
                                      train_file='train.csv')
    
label_dict = corpus.make_label_dictionary()

2019-10-25 08:04:27,063 Reading data from data
2019-10-25 08:04:27,064 Train: data\train.csv
2019-10-25 08:04:27,064 Dev: data\val.csv
2019-10-25 08:04:27,065 Test: data\test.csv
2019-10-25 08:04:27,081 Computing label dictionary. Progress:


100%|███████████████████████████████████████████████████████████████████████████████| 641/641 [00:00<00:00, 816.67it/s]


2019-10-25 08:04:41,174 [b'rec.sport.hockey', b'talk.politics.mideast', b'rec.sport.baseball', b'talk.politics.guns']


In [12]:
corpus.train[0]

Sentence: "no , boston probably wo n't go down easy , but if the sabres had n't won game one buffalo would have been out in four . and what problem are you having with playoff games here in buffalo ? ? ?!! ! ? ? ? there 's the sabres-bruins , quebec-montreal , and whatever espn and abc show , and on opposite nights there 's toronto-detroit and whatever espn (/ abc ) shows . more hockey than a good chunk of north america , either side of the border . it 's wonderful , and there are thousands of folks who 'd kill to be in your shoes , coverage-wise ." - 112 Tokens

#### Training RNN on our own dataset - Training our own model
We're going to use an RNN to run through the contextual word embeddings we got from ELMo. We will use the hidden state at the end of the document as an embedding for the entire document. We will train the RNN on our labeled dataset, so that the final hidden state carries the most relevant information for our custom classification task.  

For more information on training your own model using Flair, see [this tutorial](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md).

In [ ]:
#TODO: Change code below to fit our own dataset

In [ ]:
from flair.data import Corpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove')]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [ ]:
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('data/',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

In [ ]:
# 8. plot weight traces (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('data/weights.txt')

## Contextual Word Vectors with BERT and Stacking Embeddings
We will later use BERT, a state-of-the-art transformer model that was trained on a very large corpus and can be fine-tuned for our own custom task. The Flair package can also be used to derive contextual word embeddings using BERT and its successors. We will use a different package for BERT, to provide you with sample code for using it, and for adaptation of the weights of the BERT model itself for our own task.

Below, you can try to use BERT, Roberta, XLNet or other models provided in Flair for contextual word embeddings.  

In [ ]:
from flair.embeddings import BertEmbeddings
# init BERT
bert_embedding = BertEmbeddings('bert-base-uncased')

## Stack Embeddings in Flair
Flair also provides a simple way to stack vectors from different methods.

In [ ]:
from flair.embeddings import FlairEmbeddings, BertEmbeddings

# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# init BERT
bert_embedding = BertEmbeddings('bert-base-uncased')

In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
from flair.embeddings import StackedEmbeddings

# now create the StackedEmbedding object that combines all embeddings
stacked_embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, bert_embedding])

In [ ]:
sentence = Sentence('The grass is green .')

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

**Try it yourself:** Train a classifier using stacked embeddings of different models. Do you see an increase in performance?